In [2]:
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Embedding, LSTM
from keras.layers.core import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras import metrics, regularizers
from keras.preprocessing import sequence
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [3]:
#Load cleaned dataset
data = pd.read_csv('Cleaned_JobDescs.csv', header = 0, names = ['Query', 'Description'])

In [5]:

#Split the dataset to Training and Test subsets (90/10)
train, test = train_test_split(data, test_size = 0.1, random_state = 17) #random_state = None

train_descs = train['Description']
train_labels = train['Query']
 
test_descs = test['Description']
test_labels = test['Query']

In [6]:
# Model Parameters
vocab_size = 1000

sequences_length = 1200

embedding_dimensionality = 64 #possibly low??
max_features = 2000 #equal to vocab_size

num_labels = len(train_labels.unique())
batch_size = 32
nb_epoch = 20

nof_filters = 200
kernel_size = 16

hidden_dims = 512

In [7]:

# Convert Texts to Numeric Vectors for Input
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(train_descs)

x_train = tokenizer.texts_to_sequences(train_descs)
x_test = tokenizer.texts_to_sequences(test_descs)

x_train = sequence.pad_sequences(x_train, maxlen = sequences_length, padding = 'post')
x_test = sequence.pad_sequences(x_test, maxlen = sequences_length, padding = 'post')

encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

In [8]:
model = Sequential()
model.add(Embedding(max_features, embedding_dimensionality, input_length = 1200))
#model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))

model.add(Conv1D(nof_filters, kernel_size, padding='valid', activation='relu', strides = 1))
model.add(GlobalMaxPooling1D())

model.add(Dense(hidden_dims))
model.add(Dropout(0.3))
model.add(Activation('relu'))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', #'sgd', 'adam', 'RMSprop', 'Adagrad'
                   metrics = [metrics.categorical_accuracy])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1200, 64)          128000    
_________________________________________________________________
conv1d (Conv1D)              (None, 1185, 200)         205000    
_________________________________________________________________
global_max_pooling1d (Global (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               102912    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1

In [9]:

history = model.fit(x_train, y_train,
                    batch_size = batch_size,
                    epochs = nb_epoch,
                    verbose = True,
                    validation_split = 0.2)

Epoch 1/20
225/225 [==============================] - 143s 584ms/step - loss: 3.1601 - categorical_accuracy: 0.0637 - val_loss: 2.0208 - val_categorical_accuracy: 0.3822
Epoch 2/20
225/225 [==============================] - 130s 580ms/step - loss: 1.8429 - categorical_accuracy: 0.4253 - val_loss: 1.4427 - val_categorical_accuracy: 0.5489
Epoch 3/20
225/225 [==============================] - 131s 584ms/step - loss: 1.1501 - categorical_accuracy: 0.6382 - val_loss: 1.2295 - val_categorical_accuracy: 0.6189
Epoch 4/20
225/225 [==============================] - 134s 597ms/step - loss: 0.7294 - categorical_accuracy: 0.7734 - val_loss: 1.1599 - val_categorical_accuracy: 0.6489
Epoch 5/20
225/225 [==============================] - 128s 570ms/step - loss: 0.4768 - categorical_accuracy: 0.8559 - val_loss: 1.1792 - val_categorical_accuracy: 0.6561
Epoch 6/20
225/225 [==============================] - 113s 501ms/step - loss: 0.2854 - categorical_accuracy: 0.9224 - val_loss: 1.2179 - val_categoric

In [10]:
model.save("model.h5")

In [39]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

user_text = "Complete detailed programming and development tasks for front end public and internal websites as well as challenging back-end server code."

# Encode the text
encoded_docs = [one_hot(user_text, 500)]
# pad documents to a max length
padded_text = pad_sequences(encoded_docs, maxlen=500, padding='post')
# Prediction based on model
prediction = model.predict(padded_text)


In [40]:
# Decode the prediction
encoder = LabelBinarizer()
encoder.fit(test_labels)
result = encoder.inverse_transform(prediction)

In [41]:
print(result)

['Business Intelligence Analyst']
